In [1]:
import moleculegraph
import numpy as np
import toml

# Read FF from itp-file

This notebook allows reading foce fields from a GROMACS .itp file and saves it to a toml file that can be used for mapping with moleculegraph 

In [2]:
filename = "glycerol_gaff/Gl_gaff_new.itp"

with open(filename) as f:
    raw_lines = f.readlines()
    
lines = []
for raw in raw_lines:
    dummy = raw.split(";")
    if dummy[0]:
        line = dummy[0].split()
    else:
        line = [";"]+dummy[1].split()
    if line:
        lines.append( line )

In [3]:
def list_to_dict( list_data, key, add_branches=False ):
    dict_data = {}
    for line in list_data:
        if add_branches:
            dummy = "["+line[key]+"]"
        else:
            dummy = line[key]
        dict_data[ dummy ] = line
    return dict_data

def add_a_to_b( a,b, a_key, b_key ):
    for key in b.keys():
        b[key][b_key] = a[key][a_key]
    return b

def get_identifiers( bonds, keys, atoms_num ):
    for i,bond in enumerate(bonds):
        dummy = []
        for key in keys:
            kkey = bond[key]
            dummy.append( "["+ atoms_num[ kkey ]["name"] + "]" )
        bonds[i]["name"] = "".join( moleculegraph.general_utils.sort_force_fields( dummy ) )
    return bonds

def delete_keys( bonds, keys ):
    for i,bond in enumerate(bonds): #.keys():
        for key in keys:
            del bonds[i][key]
    return bonds

def replace_key( bonds, replace, new ):
    for i,bond in enumerate(bonds): #.keys():
        bonds[i][new] = bond[replace]
        del bonds[i][replace]
    return bonds

def add_default( bonds, add_key, add_val ):
    for bond in bonds.keys():
        bonds[bond][add_key] = add_val
    return bonds

def add_branches(bonds):
    bonds = bonds.astype(dtype='<U22').copy()
    for i,bond in enumerate(bonds):
        bonds[i] = "["+bond+"]"
    return bonds

def split_types( bonds, keys ):
    type_list = []
    names = np.array([ x["name"] for x in bonds  ])
    unames, idxs = np.unique(names, return_inverse=True)
    for i in range(len(unames)):
        p = np.where( idxs == i ) 
        dummy = np.zeros( (len(bonds[p]),len(keys))  ) #bonds[p]
        for i,bond in enumerate(bonds[p]):
            for j,key in enumerate(keys):
                dummy[i][j] = bond[key]
        dp = np.argsort( np.sum( dummy == 0, axis=-1 ) ) #np.lexsort(dummy.T)
        dummy = dummy[ dp ]
        _, pp = np.unique( dummy, return_index=True , axis=0)
        pp = np.sort(pp)
        type_list.append( bonds[p][dp][pp] )
    return type_list

In [4]:
data = {}
for i,line in enumerate(lines):
    if line[0] == "[":
        key = line[1]
        data[key] = []
        colnames = [ x.replace(",","") for x in lines[i+1][1:] ]
    elif line[0] == ";":
        None
    else:
        dummy = {}
        for col,name in zip(line,colnames):      
            try:
                a = float(col)
                b = int(col)
                if a != b:
                    xx = a
                else:
                    xx = b                    
            except:
                try:
                    xx = float(col)
                except:
                    xx = col
            dummy[name] = xx
        data[key].append(dummy)

atomtypes = np.array(data["atomtypes"])
atoms     = np.array(data["atoms"])
bonds     = np.array(data["bonds"])
angles    = np.array(data["angles"])
dihedrals = np.array(data["dihedrals"])
        
atomtypes = replace_key( atomtypes, "type", "name" )
atoms     = replace_key( atoms, "type", "name" )

atomtypes = list_to_dict( atomtypes, "name" )
atoms_num = list_to_dict( atoms.copy(), "num" )
atoms     = list_to_dict( atoms, "name" )

atomtypes = add_a_to_b( atoms,atomtypes,"q","charge" )
atomtypes = add_default( atomtypes, "cut", 14.0 )
atomtypes = add_default( atomtypes, "m", 12.0 )
atomtypes = add_default( atomtypes, "style", "lennard-jones" )        

bonds     = get_identifiers( bonds, ["ai","aj"], atoms_num )
angles    = get_identifiers( angles, ["ai","aj","ak"], atoms_num )
dihedrals = get_identifiers( dihedrals, ["i","j","k","l"], atoms_num )
bonds     = delete_keys( bonds, ["ai","aj"] )
angles    = delete_keys( angles, ["ai","aj","ak"] )
dihedrals = delete_keys( dihedrals, ["i","j","k","l"] )

dihedral_types = split_types( dihedrals, ["a","b","c","d","e","f"] )
dihedrals = []
improper_dihedrals = []
for x in dihedral_types:
    dihedrals.append(x[0])
    if len(x)>1:
        improper_dihedrals.append(x[1])
    if len(x)>2:
        print("WARNING: dihedrals not considered")
dihedrals = np.array(dihedrals)        
improper_dihedrals = np.array(improper_dihedrals) 

bonds     = list_to_dict( bonds, "name" )
angles    = list_to_dict( angles, "name" )
dihedrals = list_to_dict( dihedrals, "name" )
improper_dihedrals = list_to_dict( improper_dihedrals, "name" )

bonds     = add_default( bonds, "style", "harmonic" )
angles    = add_default( angles, "style", "harmonic" )
dihedrals = add_default( dihedrals, "style", "opls" )
improper_dihedrals = add_default( improper_dihedrals, "style", "opls" )

In [5]:
forcefield = {}
forcefield["atoms"]     = atomtypes
forcefield["bonds"]     = bonds
forcefield["angles"]    = angles
forcefield["dihedrals"] = dihedrals
forcefield["improper_dihedrals"] = improper_dihedrals

ff_file = "ff_test_GROMACS.toml"
with open(ff_file, "w") as toml_file:
    toml.dump(forcefield, toml_file)